# AppleStore and GooglePlay Profiles Applications Analysis

## Introduction

Our aim in this project is analise genres of applications to make profitable profiles that could be use in planning for next releses. Becouse our company business plan assumes making income from in-app advertising, our point of interest are only free, english applications. We are going to use data analysis to allow making data-driven decisions in this area.

To provide good sample we have avaible two data sets that came from [Kaggle](http://www.kaggle.com):
- [Google Play Store Apps](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data from around 10'000 applications from Google Play
- [Mobile App Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data from around 7'000 applications from Apple Store

In [1]:
from csv import reader

### GooglePlay Dataset ###
opened_file = open('./GooglePlay.csv')
readed_file = reader(opened_file)
google = list(readed_file)
google_header = google[0]
google = google[1:]

### AppleStore Dataset ###
opened_file = open('./AppleStore.csv')
readed_file = reader(opened_file)
apple = list(readed_file)
apple_header = apple[0]
apple = apple[1:]

# Print requested number of rows from dataset, and number of rows and columns
# if rows_and_columns = True
def explore_data(dataset, start, end, rows_and_columns = False):
    dataslice = dataset[start:end]
    
    for row in dataslice:
        print(row, '\n')
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))


## Data sets

Let's start with looking what we have in our data sets.

In [2]:
print('Google Play data:\n')
print(google_header, '\n')
explore_data(google, 0, 3, True)

Google Play data:

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

Number of rows: 10841
Number of columns: 13


As we can see in Google Play we have 10841 apps. About every application we have 13 data points. Most interesting are App, Type, Installs, Genres, Price, Rating and Reviews.

In [3]:
print('Apple Store data:\n')
print(apple_header, '\n')
explore_data(apple, 0, 3, True)

Apple Store data:

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1'] 

['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1'] 

['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1'] 

Number of rows: 7197
Number of columns: 17


In Apple Store we 7197 apps in our data set. Every application have 15 real data points (excluding ids). Most appeling seems to be track_name, prime_genre, price and ratings. Unfortunetly we don't have hery any information about number of installations.

## Data Cleaning

### Wrong record

Discussion on Kaggle outlines a wrong record in data from Google Play in row 10472. Let's see whats wrong there and compere this record with another.

In [4]:
print(google_header, 'Length:', len(google_header), '\n') # Headers
print(google[10472], 'Length:', len(google[10472]), '\n') # Wrong record
print(google[1532], 'Length:', len(google[1532]), '\n') # Correct record

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] Length: 13 

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] Length: 12 

['Essential Resources', 'LIBRARIES_AND_DEMO', '4.6', '237', '8.5k', '50,000+', 'Free', '0', 'Everyone', 'Libraries & Demo', 'September 27, 2017', '1.02', '7.1 and up'] Length: 13 



As we can see, 10472th record is one field shorter (it doesn't have Category value). We dont know correct value for category, so we'll delete this row.

In [5]:
print(len(google))
del google[10472]
print(len(google))

10841
10840


### Looking for duplicates

Now, we want to check are there any duplicated entries in the dataset. Let's start with Google Play:

In [6]:
duplicates = []
uniques = []
for app in google:
    name = app[0]
    if name in uniques:
        duplicates.append(name)
    else:
        uniques.append(name)

n_dupl_google = len(duplicates)
print(n_dupl_google)
print(duplicates[:10])

1181
['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


For example Google My Business aplication is more than two times in our dataset. We don't want to count any application more than once in our statistics, so we need to find and delete duplicated entries, but for now let's check Apple Store data.

In [7]:
duplicates = []
uniques = []
for app in apple:
    name = app[0]
    if name in uniques:
        duplicates.append(name)
    else:
        uniques.append(name)

n_dupl_apple = len(duplicates)
print(n_dupl_apple)
#print(duplicates[:10])

0


Fortunetly, Apple Store data dont have this problem. So let's go back to the Google Play data and look closely to the duplicates in Google dataset.

In [8]:
print(google_header)
for app in google:
    if app[0] == 'Slack':
        print(app)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


Only difference between entries is column 'Reviews'. We can assume that theese entries comes from diffrent hours or days, so best sollution is removing the old ones (with fewer reviews).

To do this, we need to find those entries with highest amount of Reviews, what we fulfill in the first loop. Then we go trough whole list once again and put them to the new clean list. In the end we check for sure that there is no duplicates in the list.

In [9]:
max_reviews = {}

# Go trough whole list and pick highest amount of reviews for every app 
for app in google:
    name = app[0]
    reviews = int(app[3])
    
    if name in max_reviews:
        if max_reviews[name] < reviews:
            max_reviews[name] = reviews
    else:
        max_reviews[name] = reviews
      
print('Expected length of dictionary:', len(google) - n_dupl_google)
print('Real length of dictionary:', len(max_reviews)) 

google_clean = []
already_added = []

# Then go trough list once again and add entries with highest reviews to clean list
for app in google:
    name = app[0]
    reviews = int(app[3])
    
    if (name not in already_added and reviews == max_reviews[name]):
        google_clean.append(app)
        already_added.append(name)
        
print('Clean list length:', len(google_clean))

# Check once again for duplicates to make sure everything goes as expected
duplicates = []
uniques = []
for app in google_clean:
    name = app[0]
    if name in uniques:
        duplicates.append(name)
    else:
        uniques.append(name)

print('Duplicates in clean list:', len(duplicates))

Expected length of dictionary: 9659
Real length of dictionary: 9659
Clean list length: 9659
Duplicates in clean list: 0


### Remove non-english applications

Our point of interest is only english speaking audience, we now we are going to remove all non-english app from our datasets. To provide this we are going to go trough all applications, and count number of non-english characters. Some english applications can have small number of non-english characters like question mark but if this number is higher than 3 we can assume that this application is for non-english audience.

In [10]:
# Return False if in name is more than tree non-eglish characters
# Return True otherwise
def english_only(name):
    counter = 0

    for character in name:

        if ord(character) > 127:
            counter += 1

            if counter > 3:
                return False
    
    #end of loop
    
    return True
    

eng_google = []
eng_apple = []

# We go trough every data set and if there is english name for current row we add it too new list
for app in google_clean:
    name = app[0]
    
    if english_only(name):
        eng_google.append(app)
        
#end of loop
    
for app in apple:
    name = app[2]
    
    if english_only(name):
        eng_apple.append(app)
        
#end of loop

print('Amount of english Google apps:', len(eng_google))
print('Amount of english Apple apps:', len(eng_apple))

Amount of english Google apps: 9614
Amount of english Apple apps: 6183


### Remove paid applications

Our business plan assumes main income from in-app advertising, so for our report we want count only free apps. Same as above we going to remove all paid applications from datasets.

In [11]:
free_google = []
free_apple = []

for app in eng_google:
    price = app[7]
    
    if price == '0':
        free_google.append(app)
        
# end loop

for app in eng_apple:
    price = float(app[5])
    
    if price == 0:
        free_apple.append(app)
        
# end loop

print('Amount of free, english Google apps: ', len(free_google))
print('Amount of free, english Apple apps: ', len(free_apple))

Amount of free, english Google apps:  8864
Amount of free, english Apple apps:  3222


So after all this steps we now have 8864 Google Play applications, and 3222 Apple Store applications. It might be noted, that Apple Store have bigger share of paid applications. In the next step we are going to finally analise our clean data.

In [12]:
# Returns Frequency Table from index column in dataset in form of dictionary
def freq_table(dataset, index, decimals = 2):
    
    freq = {}
    total = 0
    
    for row in dataset:
        column = row[index]
        total += 1
    
        if column in freq:
            freq[column] += 1
        else:
            freq[column] = 1

    #end of loop
    
    freq_perc = {}
    
    for key in freq:
        freq_perc[key] = 100 * ( freq[key] / total )
        freq_perc[key] = round(freq_perc[key], decimals)
    
    return freq_perc
# end of def freq_table

# Displays frequency table in descending order, cut off values below cutoff
def display_table(table, cutoff = 1):
    table_display = []
    for key in table:
        if table[key] >= cutoff:
            key_val_as_tuple = (table[key], key)
            table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

final_google = free_google
final_apple = free_apple

## Business Plan

So, our plan is find those applications genres that could provide highest income from placing in-app advertising. Income from advertising depends on number of people who watch them. So we need to find genres of applications that can provide highest number of users.

To minimalize risk, development plan assumes that we first release minimal-version of application on Google Play. If application have good responce from audience we will develop it further and add in-app advertising. If application is profitable after 6 month from adding advertizing we also make iOS version and add it to Apple Store.

Our long term goal is create application that is profitable on both markets so we need to find genres that are appropriate  on both markets. If we know what we are looking for we can start our analysis.

## Most popular genres of applications

Now we can build frequency tables for both markets. Let's recall columns headers in both datasets.

In [13]:
print(google_header, '\n')
print(apple_header, '\n')

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 



Google Play dataset have two genre related columns. It's Category and Genres, let's build frequency tables and look what we can conclude from them.

In [14]:
print('Google Play by Category:')
google_freq = freq_table(final_google, 1)
display_table(google_freq)

Google Play by Category:
FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16


In Category column we can see quite evenly distributed values. There are some applications designed for fun (Game, Photography, Lifestyle) and some disigned for practical purpose (Tools, Business, Productivity). If we look closer into Family Category we can see that it includes mostly kids games and similar applications. 

In [15]:
print('Google Play by Genres:')
google_freq2 = freq_table(final_google, -4) 
display_table(google_freq2)

Google Play by Genres:
Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13


In Genres column genres are more divided, than in Category Column. For exaple Arcade and Action are contained in Games Category. General landskape is quite similar to the previous column, and we dont need to look in every subcategory separately, so further we will use only Category column for Google Play dataset.

Apple Store have only one genre related column - prime_genre. So look what we can find here.

In [16]:
print('Apple Store by prime_genre:')
apple_freq = freq_table(final_apple, -5)
display_table(apple_freq)

Apple Store by prime_genre:
Games : 58.16
Entertainment : 7.88
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.51
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.33
Travel : 1.24
Finance : 1.12


We can clearly see that Games are most popular genre of free english applications in Apple Store by really large margin. Games genre cover over half of all applications. Second one is Entertainment that have less than 8%, followed by Photo & Video with 5%. Education covers only 3.66%.

The general impression is that the Apple Store (at least free, english part) is dominated by applications desinged for fun (Games, Entertainment, Photo & Video, Social Networking), while prictical purpose applications (Educational, Utilities, Health and Fitness, Productivity) covers really small part of the market.

## Genres of most popular applications

However number of applications in every category may vary from real amount of users that use this application. So we need to go further with our analysis. Now we are going to count average number of installations in every genre. In Google Store dateset we can do this directly, but Apple Store dataset don't include information about number of installations. Our solution to this problem is using rating_count_tot as replacement. We can assume that number of reviews depends on number of installations.

Let's start with Google Play. In out data set number of installations is given in text level form like "100,000+". This type of data isn't accurate, however it should be decent for our purpose. What we need to do is remove any non-numeric characters and convert variable to integer. 

In [17]:
# Returns number of installations as int
def google_users_fix(users):
    users = users.replace(',', '')
    users = users.replace('+', '')
    users = int(users) if users else 0
    
    return users

Now we can finally create a average users table for Google Play dataset, and look on number of installations per application in every category.

In [18]:
n_users = {}
n_app = {}

for app in final_google:
    cat = app[1]
    users = google_users_fix(app[5])
    
    if cat in n_app and cat in n_users:
        n_app[cat] += 1
        n_users[cat] += users
    else:
        n_app[cat] = 1
        n_users[cat] = users        
    

# end of loop

avg_users_google = {}

for cat in n_app:
    avg_users_google[cat] = round(n_users[cat] / n_app[cat], 2)
    
display_table(avg_users_google)

COMMUNICATION : 38456119.17
VIDEO_PLAYERS : 24727872.45
SOCIAL : 23253652.13
PHOTOGRAPHY : 17840110.4
PRODUCTIVITY : 16787331.34
GAME : 15588015.6
TRAVEL_AND_LOCAL : 13984077.71
ENTERTAINMENT : 11640705.88
TOOLS : 10801391.3
NEWS_AND_MAGAZINES : 9549178.47
BOOKS_AND_REFERENCE : 8767811.89
SHOPPING : 7036877.31
PERSONALIZATION : 5201482.61
WEATHER : 5074486.2
HEALTH_AND_FITNESS : 4188821.99
MAPS_AND_NAVIGATION : 4056941.77
FAMILY : 3695641.82
SPORTS : 3638640.14
ART_AND_DESIGN : 1986335.09
FOOD_AND_DRINK : 1924897.74
EDUCATION : 1833495.15
BUSINESS : 1712290.15
LIFESTYLE : 1437816.27
FINANCE : 1387692.48
HOUSE_AND_HOME : 1331540.56
DATING : 854028.83
COMICS : 817657.27
AUTO_AND_VEHICLES : 647317.82
LIBRARIES_AND_DEMO : 638503.73
PARENTING : 542603.62
BEAUTY : 513151.89
EVENTS : 253542.22
MEDICAL : 120550.62


The landspace here looks quite different than in the previous Google Play table. Four highest genres are Communication, Video Players, Social and Photography. This four genres was quite low in frequency table. Thats propably becouse of pre-installed applications on new devices like Messenger (Communication), YouTube (Video Players), Facebook (Social) and Gallery/Camera (Photography). In those cases will be really hard to compete with pre-installed and very popular applications.

We should look into next ones. Productivity and Game genres, was high in frequency tables, and here also have high place. We can conclude from here that saturation of the martet in those applications still have place for new products. Similar proposal we can assign for Tools genre but here numbers was a little lower.

If we look into statistics for Travel or Entertainment genres we can see that theese genres was low in frequency tables. This could mean that those type of applications are quite niche, and again it will be hard to compete with those genres stamples.

Let's take a look into Apple Store applications.

In [19]:
n_users = {}
n_app = {}

for app in final_apple:
    cat = app[-5]
    users = int(app[6])
    
    if cat in n_app and cat in n_users:
        n_app[cat] += 1
        n_users[cat] += users
    else:
        n_app[cat] = 1
        n_users[cat] = users        
    

# end of loop

avg_users_apple = {}

for cat in n_app:
    avg_users_apple[cat] = round(n_users[cat] / n_app[cat], 2)
    
display_table(avg_users_apple)

Navigation : 86090.33
Reference : 74942.11
Social Networking : 71548.35
Music : 57326.53
Weather : 52279.89
Book : 39758.5
Food & Drink : 33333.92
Finance : 31467.94
Photo & Video : 28441.54
Travel : 28243.8
Shopping : 26919.69
Health & Fitness : 23298.02
Sports : 23008.9
Games : 22788.67
News : 21248.02
Productivity : 21028.41
Utilities : 18684.46
Lifestyle : 16485.76
Entertainment : 14029.83
Business : 7491.12
Education : 7003.98
Catalogs : 4004.0
Medical : 612.0


In this table everything looks different than in the frequency table. Games and Entertainment is quite low, this propably is sign of too high number of applications in those genres. 

On the other side Navigation and Reference didn't even pass 1% of applications in frequency tables. In those genres saturation of the market could seem low, but if we look closer we will see that Google Maps covers almost all genre of Navigations app, and for Reference it's same situation with Bible and Quran.

On the first look there isn't any good niche for new application.

## Summnary and Recommendations

According to our plan (release first on Google Play, and after some success release on Apple Store), we can reccomend following genres for new products.



In [20]:
# Prints informations about given genre
def genre_info(google_name, apple_name):
    print('Percent of applications on Google Play in this genre:', google_freq[google_name])
    print('Average number of users of applications in this genre on Google Play:', avg_users_google[google_name])
    print('Percent of applications on Apple Store in this genre:', apple_freq[apple_name])
    print('Average number of ratings of applications in this genre on Apple Store:', avg_users_apple[apple_name])

### Productivity

Our first recomendation is Productivity Application. This genre market isn't exhausted on both Google Play and Apple Store. On Google Play average number of installations is over 1 500 000, and this genre covers less than 4 percent of all applications. We propably could easly find our place for new application.

On Apple Store situaction is little less optimistic but with good brand from Google Play we can also make profit there. On Apple Store this genre only covers less than 2 percent.

In [21]:
genre_info('PRODUCTIVITY', 'Productivity')

Percent of applications on Google Play in this genre: 3.89
Average number of users of applications in this genre on Google Play: 16787331.34
Percent of applications on Apple Store in this genre: 1.74
Average number of ratings of applications in this genre on Apple Store: 21028.41


### Game

The second recomendiation is Game genre. On Apple Store this genre is over 50% of market, with quite low average number of ratings but if we can create good game, with good marketing and success from Google Play we propably can also make profit on Apple Store.

On Google Play situation looks very good. There is loweer number of applications in this genre and average number of installations is high. We should easly find place in this market for new game, and with success we can expand to the Apple Store.

In [23]:
genre_info('GAME', 'Games')

Percent of applications on Google Play in this genre: 9.72
Average number of users of applications in this genre on Google Play: 15588015.6
Percent of applications on Apple Store in this genre: 58.16
Average number of ratings of applications in this genre on Apple Store: 22788.67
